In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import string
from sklearn.pipeline import Pipeline
import numpy as np
import xgboost as xgb

In [ ]:
df = pd.read_csv('Datasets/dataset_clean.csv', sep = ',',
                 encoding = 'utf-8')
df = df[['tweet', 'label']]
df

,tweet,label
0,"Soal jln Jatibaru,polisi tdk bs GERTAK gubernu...",anger
1,"Sesama cewe lho (kayaknya), harusnya bisa lebi...",anger
2,Kepingin gudeg mbarek Bu hj. Amad Foto dari go...,happy
3,"Jln Jatibaru,bagian dari wilayah Tn Abang.Peng...",anger
4,"Sharing pengalaman aja, kemarin jam 18.00 bata...",happy
...,...,...
4396,"Tahukah kamu, bahwa saat itu papa memejamkan m...",love
4397,Sulitnya menetapkan Calon Wapresnya Jokowi di ...,fear
4398,"5. masa depannya nggak jelas. lha iya, gimana ...",anger
4399,dulu beneran ada mahasiswa Teknik UI nembak p...,happy


In [ ]:
#case folding
import re
def casefolding(tweet):
    tweet = tweet.lower()
    tweet = tweet.strip(" ")
    tweet = re.sub(r'[?|$|.|!²_:")(-+,]', '', tweet)
    return tweet
df['tweet'] = df['tweet'].apply(casefolding)
df.head()

,tweet,label
0,soal jln jatibarupolisi tdk bs gertak gubernur...,anger
1,sesama cewe lho kayaknya harusnya bisa lebih r...,anger
2,kepingin gudeg mbarek bu hj amad foto dari goo...,happy
3,jln jatibarubagian dari wilayah tn abangpengat...,anger
4,sharing pengalaman aja kemarin jam 1800 batali...,happy


In [ ]:
#tokenizing
def token(tweet):
    nstr = tweet.split(' ')
    dat = []
    a = -1
    for hu in nstr:
        a = a+1
    if hu == '':
        dat.append(a)
    p = 0
    b = 0
    for q in dat :
        b = q-p
        del nstr[b]
        p = p + 1
    return nstr
df['tweet'] = df['tweet'].apply(token)
df.head()

,tweet,label
0,"[soal, jln, jatibarupolisi, tdk, bs, gertak, g...",anger
1,"[sesama, cewe, lho, kayaknya, harusnya, bisa, ...",anger
2,"[kepingin, gudeg, mbarek, bu, hj, amad, foto, ...",happy
3,"[jln, jatibarubagian, dari, wilayah, tn, abang...",anger
4,"[sharing, pengalaman, aja, kemarin, jam, 1800,...",happy


In [ ]:
#filtering
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def stopword_removal(tweet):
    filtering = stopwords.words('indonesian', 'english')
    x = []
    data = []
    def myfunc(x):
        if x in filtering:
            return False
        else:
            return True
    fit = filter(myfunc, tweet)
    for x in fit:
        data.append(x)
    return data
df['tweet'] = df['tweet'].apply(stopword_removal)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,tweet,label
0,"[jln, jatibarupolisi, tdk, bs, gertak, gubernu...",anger
1,"[cewe, lho, kayaknya, rasain, sibuk, jaga, ras...",anger
2,"[kepingin, gudeg, mbarek, bu, hj, amad, foto, ...",happy
3,"[jln, jatibarubagian, wilayah, tn, abangpengat...",anger
4,"[sharing, pengalaman, aja, kemarin, jam, 1800,...",happy


In [ ]:
!pip install Sastrawi

In [ ]:
#stemming
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(tweet):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in tweet:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean= []
    d_clean = " ".join(do)
    print(d_clean)
    return d_clean
df['tweet'] = df['tweet'].apply(stemming)
df.to_csv('datasets/data_cleanbanget.csv', index = False)

In [ ]:
data_clean = pd.read_csv('datasets/data_cleanbanget.csv', encoding = 'latin1')
data_clean.head()

,tweet,label
0,jln jatibarupolisi tdk bs gertak gubernur eman...,anger
1,cewe lho kayak rasain sibuk jaga rasain sakit ...,anger
2,kepingin gudeg mbarek bu hj amad foto google s...,happy
3,jln jatibarubagian wilayah tn abangpengaturan ...,anger
4,sharing alam aja kemarin jam 1800 batalin tike...,happy


In [ ]:
data_clean = data_clean.astype({'label': 'category'})
data_clean = data_clean.astype({'tweet': 'string'})
data_clean.dtypes

tweet      string
label    category
dtype: object